###Team Members = Tomiris, Kulash, Yussuf
***NN Model - CNN***

## Import Library

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

import sys
sys.path.insert(0,'/content/drive/MyDrive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/BMI_final_project

/content/drive/MyDrive/BMI_final_project


In [ ]:
!pip install mne==0.18.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from nu_smrutils import *

import torch 
import itertools
import pandas as pd 
import pickle 
import numpy as np
import mne

from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable 

%matplotlib inline
%load_ext autoreload
%autoreload 2
#%matplotlib qt 
torch.manual_seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Necessary functions

In [ ]:
scaler = StandardScaler()

def preprocessing(data):
  sh1=data.shape[1]
  sh2=data.shape[2]
  data=data.reshape(data.shape[0],-1)
  data=scaler.fit_transform(data)
  data=data.reshape(data.shape[0],sh1,sh2)
  return data

In [ ]:
#by default 
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if dev.type == 'cuda':
   print('Your GPU device name :', torch.cuda.get_device_name())      

Your GPU device name : Tesla T4


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

def get_data_loaders(dat, batch_size, EEGNET = None, dev=dev):    
    # convert data dimensions to into to gray scale image format
    if EEGNET: ### EEGNet model requires the last dimension to be 1 
        ff = lambda dat: torch.unsqueeze(dat, dim = -1)    
    else:
        ff = lambda dat: torch.unsqueeze(dat, dim = 1)    
        
    x_train, x_valid, x_test =  map(ff,(dat['xtrain'],dat['xvalid'],dat['xtest']))    
    y_train, y_valid, y_test =  dat['ytrain'], dat['yvalid'], dat['ytest']
    print('Input data shape', x_train.shape)       
    
    # TensorDataset & Dataloader    
    train_dat = TensorDataset(x_train.to(dev), y_train.to(dev)) 
    val_dat   = TensorDataset(x_valid.to(dev), y_valid.to(dev)) 
    
    train_loader = DataLoader(train_dat, batch_size = batch_size, shuffle = True)
    val_loader   = DataLoader(val_dat,   batch_size = batch_size, shuffle = False)

    output = dict(dset_loaders = {'train': train_loader, 'val': val_loader}, 
                  dset_sizes  =  {'train': len(x_train), 'val': len(x_valid)},
                  test_data   =  {'x_test' : x_test, 'y_test' : y_test})          
    return output 

This network we did from scratch and played with our architecture to obtain better results. So, we decided to add two Conv2d layers, and LeakyReLU function with 0.2 alpha,because it gaves better results. Also, some parameters we changed accradint to the needed task. To obtain temporal informartion we did last channels resheaping.

In [ ]:
class CNN(torch.nn.Module):  
    def __init__(self,in_shape):         
        super(CNN, self).__init__()  
        self.conv1=torch.nn.Conv2d(in_channels = 1,
                                     out_channels = 32,
                                     kernel_size = (in_shape[1],3),padding='same')
        
        self.r=nn.LeakyReLU(0.2)
        self.bn1=nn.BatchNorm2d(32)
        self.conv2=torch.nn.Conv2d(in_channels = 32,
                                     out_channels = 64,
                                     kernel_size = (in_shape[2],1),padding='same')
        self.r0=nn.LeakyReLU(0.2)
        self.bn2=nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(491520, 500)
        self.r1=nn.LeakyReLU(0.2)
        self.fc2 = nn.Linear(500, 100)
        self.r2=nn.LeakyReLU(0.2)
        self.Dropout = nn.Dropout(0.5) 
        self.fc3 = nn.Linear(100, 2)
        self.r3=nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.r(x)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.r0(x)
        x = self.bn2(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = self.r1(x)
        x=self.Dropout(x)
        x = self.fc2(x)
        x = self.r2(x)
        x = self.fc3(x)
        x = self.r3(x)
        return x

## Dataset BNCI2014004


Select Dataset

In [ ]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'BNCI2014004'

filename = dname[itemname]
iname = itemname + '__'    

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


Load subject specific data and preprocess it

In [ ]:
print('\nfilter')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 12)

#--------------------------------------------------------------------------

subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nsubject_specific')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data, with this std deviation it gaves better results 
print('\nAugment')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):    
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'], 
                                     std_dev = augmdata['std_dev'], 
                                     multiple = augmdata['multiple'])
    
    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nCropping')
fs = 80 # sampling frequency 
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'], 
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'], 
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'], 
                                    dat['ytest'], xpercent = 50)
       
       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test) 
                        
#--------------------------------------------------------------------------
# Preprocessing dataset
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=preprocessing(datt[i]['xtrain'])
  datt[i]['xvalid']=preprocessing(datt[i]['xvalid'])
  datt[i]['xtest']=preprocessing(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].reshape(datt[i]['xtrain'].shape[0],datt[i]['xtrain'].shape[2],datt[i]['xtrain'].shape[1])
  datt[i]['xvalid']=datt[i]['xvalid'].reshape(datt[i]['xvalid'].shape[0],datt[i]['xvalid'].shape[2],datt[i]['xvalid'].shape[1])
  datt[i]['xtest']=datt[i]['xtest'].reshape(datt[i]['xtest'].shape[0],datt[i]['xtest'].shape[2],datt[i]['xtest'].shape[1])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')  

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):  
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


filter


/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec


subject_specific
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Augment
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([982, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1098, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])

Cropping
torch.Size([3120, 3, 120])
torch.Size([2946, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3294, 3, 120])
tor

Training loop for dataset 

In [ ]:
#Hyperparameter settings
num_epochs = 150
learning_rate = 1e-4
weight_decay = 1e-5  
batch_size = 64
verbose = 2

fs = 80
# define kernel size in terms of ms length 
time_window = 100 #ms
width = time_window*fs//1000    

# ker = 8 #timelength//chans 
h, w = 3, 1  #hight and width of a rectangular kernel    

#% used to save the results table 
results = {}        
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch']) 
sum=0
num=0
max=0

for i in range(len(datt)): 
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2], 
                 dat['test_data']['x_test'].shape[-1])

  model = CNN(input_size)  

  # optimizer and the loss function definition 
  optimizer = torch.optim.Adam(model.parameters(), 
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)  
  criterion.to(dev)       

  #print("Model architecture >>>", model)

  #******** Training loop *********    
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'], 
                  criterion, optimizer, dev=dev, lr_scheduler=False, 
                  num_epochs = num_epochs, verbose = verbose)
          
  
  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:] 
  y_test = dat['test_data']['y_test'][:test_samples] 
  #print(x_test.shape)

  # predict test data 
  preds = best_model(x_test.to(dev)) 
  preds_class = preds.data.max(1)[1]

  # get the accuracy 
  corrects = torch.sum(preds_class == y_test.data.to(dev))     
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc) 
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results       
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],   
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)         

  table.loc[description] = tab  
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/150


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/conv.py:453: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:882.)
  return F.conv2d(input, weight, bias, self.stride,


train loss: 0.0109, acc: 0.5058
val loss: 0.0126, acc: 0.5471
Epoch 2/150
train loss: 0.0102, acc: 0.6288
val loss: 0.0124, acc: 0.5652
Epoch 3/150
train loss: 0.0096, acc: 0.6798
val loss: 0.0124, acc: 0.5906
Epoch 4/150
train loss: 0.0090, acc: 0.7272
val loss: 0.0124, acc: 0.6051
Epoch 5/150
train loss: 0.0084, acc: 0.7625
val loss: 0.0126, acc: 0.5580
Epoch 6/150
train loss: 0.0080, acc: 0.8013
val loss: 0.0119, acc: 0.6087
Epoch 7/150
train loss: 0.0077, acc: 0.8189
val loss: 0.0118, acc: 0.6123
Epoch 8/150
train loss: 0.0075, acc: 0.8298
val loss: 0.0115, acc: 0.6486
Epoch 9/150
train loss: 0.0072, acc: 0.8561
val loss: 0.0118, acc: 0.6304
Epoch 10/150
train loss: 0.0071, acc: 0.8551
val loss: 0.0113, acc: 0.6667
Epoch 11/150
train loss: 0.0069, acc: 0.8756
val loss: 0.0115, acc: 0.6522
Epoch 12/150
train loss: 0.0068, acc: 0.8840
val loss: 0.0113, acc: 0.6486
Epoch 13/150
train loss: 0.0066, acc: 0.8978
val loss: 0.0118, acc: 0.6232
Epoch 14/150
train loss: 0.0065, acc: 0.9048
v

In [ ]:
print(table)
results[description] = dict(train_accs = train_accs, val_accs =  val_accs,                                
                            ytrain = info['ytrain'], yval= info['yval'])      

fname = iname + 'CNN_POOLED' + description + '_' + str(val_acc)[:4]
torch.save(best_model.state_dict(), fname) 

print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc Epoch
Subject1   0.855128  0.666667      0.64    10
Subject2   0.486083  0.555556      0.42     1
Subject3   0.975962  0.547101      0.54    39
Subject4   0.954432  0.887719      0.94    17
Subject5   0.870787  0.649123      0.50    19
Subject6   0.968269  0.666667      0.60    26
Subject7   0.850321  0.659420      0.66    11
Subject8   0.907407  0.759450      0.76     9
Subject9   0.957051  0.677536      0.66    28

Average between subjects: 0.64
Max test_score between subjects: 0.94


## Dataset BNCI2014001


Select Dataset

In [ ]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'BNCI2014001'

filename = dname[itemname]
iname = itemname + '__'    

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


Load subject specific data and preprocess it

In [ ]:
print('\nfilter')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 12)

#--------------------------------------------------------------------------

subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nsubject_specific')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data 
print('\nAugment')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):    
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'], 
                                     std_dev = augmdata['std_dev'], 
                                     multiple = augmdata['multiple'])
    
    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nCropping')
fs = 80 # sampling frequency 
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'], 
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'], 
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'], 
                                    dat['ytest'], xpercent = 50)
       
       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test) 
                        
#--------------------------------------------------------------------------
# Preprocessing dataset
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=preprocessing(datt[i]['xtrain'])
  datt[i]['xvalid']=preprocessing(datt[i]['xvalid'])
  datt[i]['xtest']=preprocessing(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].reshape(datt[i]['xtrain'].shape[0],datt[i]['xtrain'].shape[2],datt[i]['xtrain'].shape[1])
  datt[i]['xvalid']=datt[i]['xvalid'].reshape(datt[i]['xvalid'].shape[0],datt[i]['xvalid'].shape[2],datt[i]['xvalid'].shape[1])
  datt[i]['xtest']=datt[i]['xtest'].reshape(datt[i]['xtest'].shape[0],datt[i]['xtest'].shape[2],datt[i]['xtest'].shape[1])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')    

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):  
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


filter


/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec


subject_specific
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Augment
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])

Cropping
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 

Training loop for dataset 

In [ ]:
#Hyperparameter settings
num_epochs = 80
learning_rate = 1e-4
weight_decay = 1e-4  
batch_size = 64
verbose = 2

fs = 80
# define kernel size in terms of ms length 
time_window = 100 #ms
width = time_window*fs//1000    

# ker = 8 #timelength//chans 
h, w = 3, 1  #hight and width of a rectangular kernel    

#% used to save the results table 
results = {}        
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch']) 
sum=0
num=0
max=0

for i in range(len(datt)): 
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2], 
                 dat['test_data']['x_test'].shape[-1])

  model = CNN(input_size)

  # optimizer and the loss function definition 
  optimizer = torch.optim.Adam(model.parameters(), 
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)  
  criterion.to(dev)       

  #print("Model architecture >>>", model)

  #******** Training loop *********    
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'], 
                  criterion, optimizer, dev=dev, lr_scheduler=False, 
                  num_epochs = num_epochs, verbose = verbose)
          
  
  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:] 
  y_test = dat['test_data']['y_test'][:test_samples] 
  #print(x_test.shape)

  # predict test data 
  preds = best_model(x_test.to(dev)) 
  preds_class = preds.data.max(1)[1]

  # get the accuracy 
  corrects = torch.sum(preds_class == y_test.data.to(dev))     
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc) 
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results       
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],   
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)         

  table.loc[description] = tab  
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/80
train loss: 0.0113, acc: 0.5153
val loss: 0.0135, acc: 0.4865
Epoch 2/80
train loss: 0.0093, acc: 0.7464
val loss: 0.0143, acc: 0.4685
Epoch 3/80
train loss: 0.0080, acc: 0.8196
val loss: 0.0136, acc: 0.5315
Epoch 4/80
train loss: 0.0072, acc: 0.8752
val loss: 0.0137, acc: 0.5405
Epoch 5/80
train loss: 0.0068, acc: 0.8953
val loss: 0.0142, acc: 0.5135
Epoch 6/80
train loss: 0.0064, acc: 0.9251
val loss: 0.0137, acc: 0.5045
Epoch 7/80
train loss: 0.0061, acc: 0.9477
val loss: 0.0148, acc: 0.4324
Epoch 8/80
train loss: 0.0059, acc: 0.9573
val loss: 0.0136, acc: 0.5225
Epoch 9/80
train loss: 0.0056, acc: 0.9791
val loss: 0.0140, acc: 0.5225
Epoch 10/80
train loss: 0.0055, acc: 0.9823
val loss: 0.0145, acc: 0.4865
Epoch 11/80
train loss: 0.0055, acc: 0.9815
val loss: 0.0139, acc: 0.5405
Epoch 12/80
train loss: 0.0054, acc: 0.9871
val loss: 0.0147, acc: 0.4685
Epoch 13/80
train loss: 0.0053, acc: 0.9887
val loss: 0.0142, acc: 0.4775


In [ ]:
print(table)
results[description] = dict(train_accs = train_accs, val_accs =  val_accs,                                
                            ytrain = info['ytrain'], yval= info['yval'])      

fname = iname + 'CNN_POOLED' + description + '_' + str(val_acc)[:4]
torch.save(best_model.state_dict(), fname) 

print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc Epoch
Subject1   0.999195  0.549550      0.52    49
Subject2   0.971820  0.504505      0.74    11
Subject3   0.999195  0.738739      0.70    76
Subject4   0.993559  0.576577      0.56    71
Subject5   0.975845  0.576577      0.68    12
Subject6   0.588567  0.567568      0.68     1
Subject7   0.929952  0.567568      0.50     7
Subject8   1.000000  0.630631      0.66    37
Subject9   0.996779  0.711712      0.80    56

Average between subjects: 0.65
Max test_score between subjects: 0.80


## Dataset Weibo2014


Select Dataset

In [ ]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'Weibo2014'

filename = dname[itemname]
iname = itemname + '__'    

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 10


Load subject specific data and preprocess it

In [ ]:
print('\nfilter')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 12)

#--------------------------------------------------------------------------

subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nsubject_specific')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data 
print('\nAugment')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):    
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'], 
                                     std_dev = augmdata['std_dev'], 
                                     multiple = augmdata['multiple'])
    
    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nCropping')
fs = 80 # sampling frequency 
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'], 
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'], 
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'], 
                                    dat['ytest'], xpercent = 50)
       
       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test) 
                        
#--------------------------------------------------------------------------
# Preprocessing dataset
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=preprocessing(datt[i]['xtrain'])
  datt[i]['xvalid']=preprocessing(datt[i]['xvalid'])
  datt[i]['xtest']=preprocessing(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].reshape(datt[i]['xtrain'].shape[0],datt[i]['xtrain'].shape[2],datt[i]['xtrain'].shape[1])
  datt[i]['xvalid']=datt[i]['xvalid'].reshape(datt[i]['xvalid'].shape[0],datt[i]['xvalid'].shape[2],datt[i]['xvalid'].shape[1])
  datt[i]['xtest']=datt[i]['xtest'].reshape(datt[i]['xtest'].shape[0],datt[i]['xtest'].shape[2],datt[i]['xtest'].shape[1])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')   

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):  
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


filter


/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec


subject_specific
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
Loading subject: 10
list index out of range

Augment
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([202, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])

Cropping
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torc

Training loop for dataset 

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
#Hyperparameter settings,we tried to played with it to obtain better results,and here final version
num_epochs = 50
learning_rate = 1e-4
weight_decay = 1e-4  
batch_size = 64
verbose = 2

fs = 80
# define kernel size in terms of ms length 
time_window = 100 #ms
width = time_window*fs//1000    

# ker = 8 #timelength//chans 
h, w = 3, 1  #hight and width of a rectangular kernel    

#% used to save the results table 
results = {}        
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch']) 
sum=0
num=0
max=0

for i in range(len(datt)): 
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2], 
                 dat['test_data']['x_test'].shape[-1])

  model = CNN(input_size)

  # optimizer and the loss function definition 
  optimizer = torch.optim.Adam(model.parameters(), 
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)  
  criterion.to(dev)       

  #print("Model architecture >>>", model)

  #******** Training loop *********    
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'], 
                  criterion, optimizer, dev=dev, lr_scheduler=False, 
                  num_epochs = num_epochs, verbose = verbose)
          
  
  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:] 
  y_test = dat['test_data']['y_test'][:test_samples] 
  #print(x_test.shape)

  # predict test data 
  preds = best_model(x_test.to(dev)) 
  preds_class = preds.data.max(1)[1]

  # get the accuracy 
  corrects = torch.sum(preds_class == y_test.data.to(dev))     
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc) 
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results       
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],   
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)         

  table.loc[description] = tab  
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/50
train loss: 0.0117, acc: 0.5391
val loss: 0.0115, acc: 0.5397
Epoch 2/50
train loss: 0.0092, acc: 0.7319
val loss: 0.0113, acc: 0.5238
Epoch 3/50
train loss: 0.0079, acc: 0.8290
val loss: 0.0118, acc: 0.5079
Epoch 4/50
train loss: 0.0072, acc: 0.8768
val loss: 0.0114, acc: 0.5556
Epoch 5/50
train loss: 0.0066, acc: 0.9072
val loss: 0.0116, acc: 0.5556
Epoch 6/50
train loss: 0.0063, acc: 0.9449
val loss: 0.0124, acc: 0.4921
Epoch 7/50
train loss: 0.0060, acc: 0.9507
val loss: 0.0125, acc: 0.4921
Epoch 8/50
train loss: 0.0057, acc: 0.9623
val loss: 0.0126, acc: 0.4921
Epoch 9/50
train loss: 0.0055, acc: 0.9739
val loss: 0.0128, acc: 0.4762
Epoch 10/50
train loss: 0.0054, acc: 0.9826
val loss: 0.0125, acc: 0.4762
Epoch 11/50
train loss: 0.0053, acc: 0.9870
val loss: 0.0120, acc: 0.5238
Epoch 12/50
train loss: 0.0053, acc: 0.9841
val loss: 0.0125, acc: 0.5238
Epoch 13/50
train loss: 0.0052, acc: 0.9928
val loss: 0.0123, acc: 0.5238


In [ ]:
print(table)
results[description] = dict(train_accs = train_accs, val_accs =  val_accs,                                
                            ytrain = info['ytrain'], yval= info['yval'])      

fname = iname + 'CNN_POOLED' + description + '_' + str(val_acc)[:4]
torch.save(best_model.state_dict(), fname) 

print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

           Train_Acc   Val_Acc  Test_Acc Epoch
Subject1    0.876812  0.555556      0.46     4
Subject2    0.921739  0.571429      0.44     4
Subject3    0.813043  0.650794      0.56     2
Subject4    0.927536  0.587302      0.40     7
Subject5    0.946377  0.603175      0.48     5
Subject6    0.963696  0.740741      0.56    10
Subject7    0.995652  0.873016      0.68     9
Subject8    0.991304  0.634921      0.48     9
Subject9    0.979710  0.571429      0.60    23
Subject10   0.523188  0.539683      0.52     1

Average between subjects: 0.52
Max test_score between subjects: 0.68


## Dataset Physionet


Select Dataset

In [ ]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'Physionet'

filename = dname[itemname]
iname = itemname + '__'    

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 109


Load subject specific data and preprocess it

In [ ]:
print('\nfilter')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 12)

#--------------------------------------------------------------------------

subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nsubject_specific')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data 
print('\nAugment')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):    
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'], 
                                     std_dev = augmdata['std_dev'], 
                                     multiple = augmdata['multiple'])
    
    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nCropping')
fs = 80 # sampling frequency 
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'], 
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'], 
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'], 
                                    dat['ytest'], xpercent = 50)
       
       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test) 
                        
#--------------------------------------------------------------------------
# Preprocessing dataset
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=preprocessing(datt[i]['xtrain'])
  datt[i]['xvalid']=preprocessing(datt[i]['xvalid'])
  datt[i]['xtest']=preprocessing(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].reshape(datt[i]['xtrain'].shape[0],datt[i]['xtrain'].shape[2],datt[i]['xtrain'].shape[1])
  datt[i]['xvalid']=datt[i]['xvalid'].reshape(datt[i]['xvalid'].shape[0],datt[i]['xvalid'].shape[2],datt[i]['xvalid'].shape[1])
  datt[i]['xtest']=datt[i]['xtest'].reshape(datt[i]['xtest'].shape[0],datt[i]['xtest'].shape[2],datt[i]['xtest'].shape[1])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')   

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):  
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


filter
Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 133 samples (1.663 sec)



/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec

Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 133 samples (1.663 sec)



/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec

Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 133 samples (1.663 sec)



...

Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 133 samples (1.663 sec)



/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec

Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 133 samples (1.663 sec)



/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec


subject_specific
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
Loading subject: 10
Loading subject: 11
Loading subject: 12
Loading subject: 13
Loading subject: 14
Loading subject: 15
Loading subject: 16
Loading subject: 17
Loading subject: 18
Loading subject: 19
Loading subject: 20
Loading subject: 21
Loading subject: 22
Loading subject: 23
Loading subject: 24
Loading subject: 25
Loading subject: 26
Loading subject: 27
Loading subject: 28
Loading subject: 29
Loading subject: 30
Loading subject: 31
Loading subject: 32
Loading subject: 33
Loading subject: 34
Loading subject: 35
Loading subject: 36
Loading subject: 37
Loading subject: 38
Loading subject: 39
Loading subject: 40
Loading subject: 41
Loading subject: 42
Loading subject: 43
Loading subject: 44
Loading subject: 45
Loading subject: 46
Loading subject: 47
Loading subject: 48
Loading subj

Training loop for dataset 

In [ ]:
#Hyperparameter settings
num_epochs = 50
learning_rate = 1e-3
weight_decay = 1e-4  
batch_size = 64
verbose = 2

fs = 80
# define kernel size in terms of ms length 
time_window = 100 #ms
width = time_window*fs//1000    

# ker = 8 #timelength//chans 
h, w = 3, 1  #hight and width of a rectangular kernel    

#% used to save the results table 
results = {}        
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch']) 
sum=0
num=0
max=0

for i in range(len(datt)): 
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2], 
                 dat['test_data']['x_test'].shape[-1])

  model = CNN(input_size)

  # optimizer and the loss function definition 
  optimizer = torch.optim.Adam(model.parameters(), 
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)  
  criterion.to(dev)       

  #print("Model architecture >>>", model)

  #******** Training loop *********    
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'], 
                  criterion, optimizer, dev=dev, lr_scheduler=False, 
                  num_epochs = num_epochs, verbose = verbose)
          
  
  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:] 
  y_test = dat['test_data']['y_test'][:test_samples] 
  #print(x_test.shape)

  # predict test data 
  preds = best_model(x_test.to(dev)) 
  preds_class = preds.data.max(1)[1]

  # get the accuracy 
  corrects = torch.sum(preds_class == y_test.data.to(dev))     
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc) 
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results       
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],   
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)         

  table.loc[description] = tab  
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

Streaming output truncated to the last 5000 lines.
train loss: 0.0091, acc: 0.7586
val loss: 0.0493, acc: 0.7500
Epoch 20/50
train loss: 0.0093, acc: 0.7586
val loss: 0.0479, acc: 0.7500
Epoch 21/50
train loss: 0.0098, acc: 0.7155
val loss: 0.0410, acc: 0.7500
Epoch 22/50
train loss: 0.0091, acc: 0.7500
val loss: 0.0410, acc: 0.8333
Epoch 23/50
train loss: 0.0095, acc: 0.7500
val loss: 0.0462, acc: 0.8333
Epoch 24/50
train loss: 0.0094, acc: 0.7672
val loss: 0.0549, acc: 0.6667
Epoch 25/50
train loss: 0.0095, acc: 0.7500
val loss: 0.0661, acc: 0.5833
Epoch 26/50
train loss: 0.0096, acc: 0.7241
val loss: 0.0661, acc: 0.5833
Epoch 27/50
train loss: 0.0104, acc: 0.7155
val loss: 0.0704, acc: 0.5000
Epoch 28/50
train loss: 0.0101, acc: 0.7241
val loss: 0.0704, acc: 0.5000
Epoch 29/50
train loss: 0.0100, acc: 0.7328
val loss: 0.0661, acc: 0.5000
Epoch 30/50
train loss: 0.0097, acc: 0.7414
val loss: 0.0618, acc: 0.5000
Epoch 31/50
train loss: 0.0098, acc: 0.7414
val loss: 0.0592, acc: 0.6667

In [ ]:
print(table)
results[description] = dict(train_accs = train_accs, val_accs =  val_accs,                                
                            ytrain = info['ytrain'], yval= info['yval'])      

fname = iname + 'CNN_POOLED' + description + '_' + str(val_acc)[:4]
torch.save(best_model.state_dict(), fname) 

print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

            Train_Acc   Val_Acc  Test_Acc Epoch
Subject1     0.625000  0.583333  0.500000     1
Subject2     0.646552  0.583333  0.642857    17
Subject3     0.757812  0.833333  0.571429    12
Subject4     0.801724  0.750000  0.428571     6
Subject5     0.750000  0.500000  0.500000    19
...               ...       ...       ...   ...
Subject104   0.589286  0.666667  0.416667     1
Subject105   0.681034  0.833333  0.428571    50
Subject106   0.765625  0.750000  0.571429     2
Subject107   0.429688  0.666667  0.642857     1
Subject108   0.836207  0.833333  0.642857     8

[108 rows x 4 columns]

Average between subjects: 0.50
Max test_score between subjects: 0.71


In [ ]:
table[:]

,Train_Acc,Val_Acc,Test_Acc,Epoch
Subject1,0.625000,0.583333,0.500000,1
Subject2,0.646552,0.583333,0.642857,17
Subject3,0.757812,0.833333,0.571429,12
Subject4,0.801724,0.750000,0.428571,6
Subject5,0.750000,0.500000,0.500000,19
...,...,...,...,...
Subject104,0.589286,0.666667,0.416667,1
Subject105,0.681034,0.833333,0.428571,50
Subject106,0.765625,0.750000,0.571429,2
Subject107,0.429688,0.666667,0.642857,1
